# Vectorstore Agent

这个笔记本展示了一个代理，旨在检索一个或多个Vectorstore的信息，带有或不带有来源。

## 创建Vectorstores

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
llm = OpenAI(temperature=0)

In [3]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


## 初始化工具包和Agent

首先，我们将创建一个带有单个vectorstore的代理。

In [4]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="the most recent state of the Union adress",
    vectorstore=state_of_union_store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

## 实例

In [5]:
agent_executor.run("What did biden say about ketanji brown jackson in the state of the union address?")



> Entering new AgentExecutor chain...
 I need to find the answer in the state of the union address
Action: state_of_union_address
Action Input: What did biden say about ketanji brown jackson
Observation:  Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

In [6]:
agent_executor.run("What did biden say about ketanji brown jackson in the state of the union address? List the source.")



> Entering new AgentExecutor chain...
 I need to use the state_of_union_address_with_sources tool to answer this question.
Action: state_of_union_address_with_sources
Action Input: What did biden say about ketanji brown jackson
Observation: {"answer": " Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.\n", "sources": "../../state_of_the_union.txt"}
Thought: I now know the final answer
Final Answer: Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. Sources: ../../state_of_the_union.txt

> Finished chain.


"Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. Sources: ../../state_of_the_union.txt"

## 多个Vectorstore
我们还可以轻松使用此功能，初始化具有多个Vectorstore的代理，并使用代理在它们之间进行路由。为此，此代理经过了优化，因此它是不同的工具包和初始化程序。

In [7]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_router_agent,
    VectorStoreRouterToolkit,
    VectorStoreInfo,
)

In [8]:
ruff_vectorstore_info = VectorStoreInfo(
    name="ruff",
    description="Information about the Ruff python linting library",
    vectorstore=ruff_store
)
router_toolkit = VectorStoreRouterToolkit(
    vectorstores=[vectorstore_info, ruff_vectorstore_info],
    llm=llm
)
agent_executor = create_vectorstore_router_agent(
    llm=llm,
    toolkit=router_toolkit,
    verbose=True
)

## 实例

In [9]:
agent_executor.run("What did biden say about ketanji brown jackson in the state of the union address?")



> Entering new AgentExecutor chain...
 I need to use the state_of_union_address tool to answer this question.
Action: state_of_union_address
Action Input: What did biden say about ketanji brown jackson
Observation:  Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

In [10]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks?")



> Entering new AgentExecutor chain...
 I need to find out what tool ruff uses to run over Jupyter Notebooks
Action: ruff
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. After installing ruff and nbqa, you can run Ruff over a notebook like so: > nbqa ruff Untitled.ipynb
Thought: I now know the final answer
Final Answer: Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. After installing ruff and nbqa, you can run Ruff over a notebook like so: > nbqa ruff Untitled.ipynb

> Finished chain.


'Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. After installing ruff and nbqa, you can run Ruff over a notebook like so: > nbqa ruff Untitled.ipynb'

In [11]:
agent_executor.run("What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?")



> Entering new AgentExecutor chain...
 I need to find out what tool ruff uses and if the president mentioned it in the state of the union.
Action: ruff
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff is integrated into nbQA, a tool for running linters and code formatters over Jupyter Notebooks. After installing ruff and nbqa, you can run Ruff over a notebook like so: > nbqa ruff Untitled.ipynb
Thought: I need to find out if the president mentioned nbQA in the state of the union.
Action: state_of_union_address
Action Input: Did the president mention nbQA in the state of the union?
Observation:  No, the president did not mention nbQA in the state of the union.
Thought: I now know the final answer.
Final Answer: No, the president did not mention nbQA in the state of the union.

> Finished chain.


'No, the president did not mention nbQA in the state of the union.'